# Facility Fit Tester
### Pick estimates of unit operations or select from type drop down to autopopulate

In [ ]:
# from platform import python_version 

# python_version()



In [13]:
# import pkg_resources
# import types
# def get_imports():
#     for name, val in globals().items():
#         if isinstance(val, types.ModuleType):
#             # Split ensures you get root package, 
#             # not just imported function
#             name = val.__name__.split(".")[0]

#         elif isinstance(val, type):
#             name = val.__module__.split(".")[0]

#         # Some packages are weird and have different
#         # imported names vs. system/pip names. Unfortunately,
#         # there is no systematic way to get pip names from
#         # a package's imported name. You'll have to add
#         # exceptions to this list manually!
#         poorly_named_packages = {
#             "PIL": "Pillow",
#             "sklearn": "scikit-learn"
#         }
#         if name in poorly_named_packages.keys():
#             name = poorly_named_packages[name]

#         yield name
# imports = list(set(get_imports()))

# # The only way I found to get the version of the root package
# # from only the name of the package is to cross-check the names 
# # of installed packages vs. imported packages
# requirements = []
# for m in pkg_resources.working_set:
#     if m.project_name in imports and m.project_name!="pip":
#         requirements.append((m.project_name, m.version))

# for r in requirements:
#     print("{}=={}".format(*r))

In [ ]:

from dateutil.relativedelta import relativedelta
import sys 
from ipywidgets import HBox, VBox, Text, Button, Layout, GridBox, Dropdown,RadioButtons,AppLayout,GridspecLayout,Label
from ipywidgets import *
import numpy as np
import pandas as pd
from datetime import datetime
import requests
import json
import requests
import datetime
import webbrowser
import time
from scipy import stats

import ipysheet
import ipyaggrid
import qgrid

In [11]:

#make dictionary of states and codes 
Types=['mAb','Recombinant virus','Transfection','Adherent','Fermentation','E Coli','Yeast']



class NOAA_APP2():
    def __init__(self):
        #initialize buttons 
        layout={'border': '1px solid black'} 
        style = {'description_width': '200px',}
        layout = {'width:':'4000px'}
        self.docButton= widgets.Button(description='NOAA Documentation',style=style)
        self.states=widgets.Dropdown(options=Types,description='State',layout = {'width:':'100px'})
        self._button = widgets.Button(description="Query NOAA",style=style, layout=layout)
        self.radio = widgets.RadioButtons(options=['Date Range','Single Date'],value='Date Range',layout = {'width:':'200px'})
        #start and end default to dates i know have data 
        self.STARTDATE=widgets.DatePicker( description='Start Date', disabled=False,value=datetime.datetime(2019,5,23))
        self.ENDDATE=widgets.DatePicker( description='End Date', value=datetime.datetime(2019,6,1),disabled=False)
        #years you can iterate over for single date 
        self.years = widgets.Dropdown(options=np.linspace(1,5,5),description='Years');self.years.layout.visibility = 'hidden'
        #output widgets 
        self.out =widgets.Output(layout=layout)
        self.out2 =widgets.Output( layout=layout)
        self.out3 = widgets.Output(layout=layout);self.out4=widgets.Output(layout=layout)
        self.about = widgets.Output(layout=layout)
       
        self.status= widgets.Label(style={'description_width': '1000px'})
        # add tabs 
        self.tab = Tab(layout=Layout(height='700px'))
        self.tab.children = [self.about,self.out,self.out2,self.out3,self.out4]
        #tab names 
        tabs = ['About App','Stations Map','Stations Data','Weather Data','Weather Table']
        #for each tab have this header 
        for x in range(0,len(tabs)): 
            self.tab.set_title(x,tabs[x])
        self.header=VBox([HBox([self.radio,self.years,self.states,self.STARTDATE,self.ENDDATE]),
                          HBox([self._button,self.docButton,self.status])]) 
        self.container=VBox([self.header,self.tab])
        #description of app use 
        with self.about:
            print('Pick estimates of unit operations or select from type drop down to autopopulate')
    
              #   'See NOAA documentation button for weather type abbreviations'+'\033[1m \n\n\nQUERY HISTORY\n\n \033[0m' )
    def on_click(self, function): #get data button
        self._button.on_click(self.get_data)
    def on_click_doc(self,button): ##documentation on GHCND 
        DOC='https://www1.ncdc.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf'
        webbrowser.open(DOC)
#     def DSclick(self,function): #get datasets 
#         self.status.value='Retreiving Datasets...please wait'
#         self.DataSet.options = self.get_DATAsets(self.zipp.value)
#         self.status.value='Dropdown populated....select dataset and query NOAA API'
    def radio_called(self,function): #radio button for single date or range 
        if self.radio.value == 'Single Date':
            self.ENDDATE.layout.visibility = 'hidden'
            self.STARTDATE.description='Date'
            self.years.layout.visibility = 'visible'
        else:
            self.ENDDATE.layout.visibility = 'visible'
            self.years.layout.visibility = 'hidden'
            self.STARTDATE.description='Start Date'
     
    def get_data(self,function):
        self.status.value='2'
#         #get statsions 
#         self.status.value='Retreiving stations...please wait'
#         df = self.get_stations()
#          #clear output
#         self.out.clear_output(); self.out2.clear_output(); self.out3.clear_output(); self.out4.clear_output()
#         if len(df)!=0: #if we return stations
#             self.status.value='Stations found'
#             with self.out: #show map of stations 
#                 fig = go.Figure(data=go.Scattergeo(lon = df['longitude'], lat = df['latitude'],
#                         text = df['name'],mode = 'markers',         ))
#                 fig.update_layout(title = str(self.states.value)+'Stations with Data',geo_scope='usa' ); display(fig)
#                 self.tab.children=list(self.tab.children)
#             with self.out2: #print dataframe to tab of weather stations 
#                 s=ipysheet.from_dataframe(df)
#                 display(df)
#             self.status.value="Retrieved "+str(len(df))+" stations from NOAA within date range"
#             self.stationList =  df['id']
#             self.status.value='getting weather....please wait'
#             w= self.get_weather(self.stationList)
#             #if there is weather data 
#             if len(w)!=0:        
#                 self.status.value="Successfully retrieved "+str(len(w))+" weather records"   
#                 with self.out3: #scatter by type 
#                     fig2 = go.Figure(data=px.scatter(w,x='date',y='value',color='datatype' ))
#                     fig2.update_layout(title = 'Weather Data',xaxis_tickformat = '%d %B <br> %Y' ); display(fig2)
#                     display(w.groupby('datatype').describe())
#                 with self.out4:
#                     sheet=ipysheet.from_dataframe(w)
#                     display(sheet)
#                     self.tab.children = list(self.tab.children)
#             else:
#                 self.status.value='No weather for date range'


    def get_stations(self): 
        self.status.value='1'
#         token = {'token': "BIUxZcnpFtJJmupYRZVElaHklugqXxNb"}
#         if self.radio.value == 'Single Date': 
#                 total = {}
#                 Dy = self.get_dates() #for each date for years 
#                 for date in Dy:
#                     self.status.value='Retreiving stations for '+ date+'...please wait'
#                     params = ('datasetid=GHCND&locationid='+DICT[str(self.states.value)]+'&startdate='+date+
#                               '&limit=10'+'&sortfield=mindate')
#                     r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?',params =params, headers=token)
#                     #exit if rate limit reached 
#                     if r.status_code==429:
#                         self.status.value='Rate limit exceeded';sys.exit()
#                     #print("Request status code: "+str(r.status_code))
#                     try:
#                         total[date]= pd.json_normalize(r.json(),record_path =['results'])    
        
#                     except:
#                         self.status.valuev='No available stations for date range'
#                         return pd.DataFrame() 
#                 #return data frame of stations 
#                 df=pd.concat(total.values(), ignore_index=True).drop_duplicates(subset=['id'])
#                 return df 

#         else: #for date range 
#             params = ('datasetid=GHCND&locationid='+DICT[str(self.states.value)]+
#                       '&startdate='+str(self.STARTDATE.value)[0:10]+'&enddate='+str(self.ENDDATE.value)[0:10]+
#                       '&limit=50'+'&sortfield=mindate')
#             r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/stations?',params =params, headers=token)
#             #if rate limit exit 
#             if r.status_code==429:
#                 self.status.value='Rate limit exceeded';sys.exit()
#             #print("Request status code: "+str(r.status_code))
#             try:
#                 df= pd.json_normalize(r.json(),record_path =['results'])    
#                 return  df
#             except: #return dataframe or empty 
#                 self.status.value='No available stations for date range'
#                 return pd.DataFrame() 

#     def get_dates(self):
#         Dy=[] #get each year data for single date 
#         for x in range(0,int(self.years.value)):
#             Dy.append(str(self.STARTDATE.value+ relativedelta(years=-x))[0:10])
#         return Dy
    
#     def get_weather(self,stationlist):#,datatype):
#         token = {'token': "BIUxZcnpFtJJmupYRZVElaHklugqXxNb"} 
#         string = ''   
#         for x in stationlist: #make station list for query 
#             string=string+'&stationid='+x
#     #if single date 
#         if self.radio.value=='Single Date':
#             Dy = self.get_dates();total = {}
#             with self.about: #query for GHCND data 
#                 print(str(self.states.value)+' '+str(Dy)+'\n')
#             for date in Dy: #for each year 
#                 self.status.value='Retreiving weather for '+ date+'...please wait'
#                 params = ('datasetid=GHCND'+string+'&startdate='+
#                       date+'&enddate='+date+'&limit=200'+'&'+'units=standard')
              
#                 r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?', params = params, headers=token)
#              #exit if rate limit 
#                 if r.status_code==429:
#                     self.status.value='Rate limit exceeded';sys.exit() 
#                 try:          
#                     total[date]= pd.json_normalize(r.json(),record_path =['results'])   
#                 except:
#                     self.status.value='No available weather your date range'
#                     return pd.DataFrame() 
#             df=pd.concat(total.values(), ignore_index=True)
#             return df  
#         #for data range do this 
#         else:
#             with self.about:
#                 print(str(self.states.value)+' '+ str(self.STARTDATE.value)[0:10]+' '+str(self.ENDDATE.value)[0:10]+'\n')
#             params = ('datasetid=GHCND'+string+'&startdate='+ str(self.STARTDATE.value)[0:10]+'&enddate='+
#                       str(self.ENDDATE.value)[0:10]+ '&limit=1000'+'&'+'units=standard')
#         r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?', params = params, headers=token)
#         if r.status_code==429:
#             self.status.value='Rate limit exceeded';sys.exit() #print("Request status code: "+str(r.status_code))
#         try: #return dataframe or empty 
#             df= pd.json_normalize(r.json(),record_path =['results'])
#             return df 
#         except:
#             print("No weather for date range and location")
#             return pd.DataFrame()
            
    
    def get_layout(self):
        return self.container
setup = NOAA_APP2()

In [12]:
def on_setup(btn): #setup tab layout 
    setup.get_data()
#query button, documention button, radio button    
setup._button.on_click(setup.on_click)
setup.docButton.on_click(setup.on_click_doc)
setup.radio.observe(setup.radio_called)

setup.get_layout()
